In [ ]:
from datasets import Dataset 
import pandas as pd
from datasets import load_dataset
dataset = load_dataset("parquet", data_files="../data/dataset_big.parquet")["train"]

Generating train split: 0 examples [00:00, ? examples/s]

In [8]:
tokenized_dataset = dataset

In [9]:
from transformers import AutoModelForCausalLM
from transformers import GPT2Tokenizer
model = AutoModelForCausalLM.from_pretrained(
    "openai-community/gpt2",
    device_map="auto",
    low_cpu_mem_usage=True
    
)

c:\Users\JACK\AppData\Local\Programs\Python\Python310\lib\site-packages\accelerate\utils\modeling.py:1569: UserWarning: Current model requires 1572912.0 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


In [10]:
from peft import get_peft_model, LoraConfig, TaskType

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
)

model = get_peft_model(model, peft_config)

c:\Users\JACK\AppData\Local\Programs\Python\Python310\lib\site-packages\peft\tuners\lora\layer.py:1768: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [11]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./gpt2-medical-lora",
    per_device_train_batch_size=8,
    num_train_epochs=3,
    learning_rate=2e-4,
    logging_steps=50,
    save_steps=500,
    save_total_limit=2,
    report_to="none",        
    gradient_accumulation_steps=4,
)

trainer = Trainer(   
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)
trainer.train()


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
50,3.700500
100,3.144300
150,3.048300
200,2.959000
250,2.969100
300,2.912900
350,2.891000
400,2.872300
450,2.890900
500,2.847500


KeyboardInterrupt: 